In [1]:
import keras
import numpy as np
from colorist import BgColorRGB
from keras.src.utils import load_img, img_to_array

def rgb_to_tuple(rgb):
    return (
        int(min(max(rgb[0] * 255,0),255)), int(min(max(rgb[1] * 255,0),255)), int(min(max(rgb[2] * 255,0),255))
    )

model = keras.models.load_model('model.keras')

def hex_to_rgb(hex_color):
    return [int(hex_color[i:i+2], 16)/255.0 for i in (0, 2, 4)]

def f(t):
    delta = 6/29
    return np.where(t > delta**3, np.cbrt(t), (t / (3 * delta**2)) + (4/29))

def inv_f(t):
    delta = 6/29
    return np.where(t > delta, t**3, 3 * delta**2 * (t - 4/29))

def inv_gamma_correct(c):
    return np.where(c <= 0.0031308, 12.92 * c, 1.055 * np.power(c, 1/2.4) - 0.055)

def lab_normalize(lab):
    return (lab + np.array([0, 128, 128])) / np.array([100, 255, 255])

def lab_unnorm(lab):
    return lab * np.array([100, 255, 255]) - np.array([0, 128, 128])

def lab_to_rgb(lab):
    L, a, b = lab
    fy = (L + 16) / 116
    fx = fy + a / 500
    fz = fy - b / 200
    xyz = np.array([
        0.95047 * inv_f(fx),
        1.00000 * inv_f(fy),
        1.08883 * inv_f(fz)
    ])
    rgb_lin = np.array([
        3.2406 * xyz[0] - 1.5372 * xyz[1] - 0.4986 * xyz[2],
        -0.9689 * xyz[0] + 1.8758 * xyz[1] + 0.0415 * xyz[2],
        0.0557 * xyz[0] - 0.2040 * xyz[1] + 1.0570 * xyz[2]
    ])
    rgb = inv_gamma_correct(np.clip(rgb_lin, 0, 1))
    return np.clip(rgb, 0, 1)

def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))

def convert_lab(image):
    mask = image > 0.04045
    img_linear = np.where(mask, ((image + 0.055) / 1.055) ** 2.4, image / 12.92)
    R, G, B = img_linear[..., 0], img_linear[..., 1], img_linear[..., 2]
    X = (0.4124564 * R + 0.3575761 * G + 0.1804375 * B) / 0.950489
    Y = (0.2126729 * R + 0.7151522 * G + 0.0721750 * B) / 1.0
    Z = (0.0193339 * R + 0.1191920 * G + 0.9503041 * B) / 1.088840
    X, Y, Z = f(X), f(Y), f(Z)
    L = 116.0 * Y - 16.0
    a = 500.0 * (X - Y)
    b = 200.0 * (Y - Z)
    return np.stack([L, a, b], axis=-1)



2025-06-11 19:36:12.104912: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-11 19:36:12.107122: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-11 19:36:12.113466: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749663372.123970   21092 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749663372.127168   21092 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749663372.136012   21092 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
image_path = "/home/drabcio/Pobrane/OneDrive_1_26.04.2025/PhotosColorPicker/000000203107.jpg"
image = load_img(image_path, target_size=(64, 64))
img_array = img_to_array(image) / 255.0

#lab
# lab_image = convert_lab(img_array)
# lab_normed = lab_normalize(lab_image)
# input_arr = np.expand_dims(lab_normed, axis=0)
# prediction = model.predict(input_arr)[0]
#
#rgb
#input_arr = np.expand_dims(img_array, axis=0)
#prediction = model.predict(input_arr)[0]
input_arr = np.expand_dims(img_array, axis=0)
prediction = model.predict(input_arr)[0]
lab_pred = lab_unnorm(prediction)
rgb_pred = lab_to_rgb(lab_pred)
pred_hex = rgb_to_hex(rgb_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


ValueError: operands could not be broadcast together with shapes (15,) (3,) 

In [3]:
rgb = rgb_to_tuple(rgb_pred)
color = BgColorRGB(rgb[0], rgb[1], rgb[2])

print("lab:",lab_pred)
print("Predicted color:", pred_hex)
print(f"{color}   {color.OFF}")
print(f"{color}   {color.OFF}")
print(f"{color}   {color.OFF}")

lab: [ 68.84066463 -21.33372727  17.21031767]
Predicted color: #8bb188
   
   
   
